In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

# Custom Imports
import wrangle

#### Exercises part 1 using telco

Acquire and prep 

### Exercise part 2 using Zillow

In [3]:
df = wrangle.wrangle_zillow()
df.head(5)

NameError: name 'np' is not defined